# 1. Problem Framing

Business application is that the company identified a lot of wastage in the business. To reduce this we are determining when extra staff are needed and when they are not needed.

# 2. Exploratory Data Analysis

In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [89]:
df = pd.read_csv('H2.csv')

In [90]:
df.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,6,2015,July,27,1,0,2,1,0.0,...,No Deposit,6,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03
1,1,88,2015,July,27,1,0,4,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01
2,1,65,2015,July,27,1,0,4,1,0.0,...,No Deposit,9,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30
3,1,92,2015,July,27,1,2,4,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23
4,1,100,2015,July,27,2,0,2,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02


In [91]:
len(df)

79330

In [92]:
df.IsCanceled.value_counts()

0    46228
1    33102
Name: IsCanceled, dtype: int64

In [93]:
df_null = df[df['Company'] == '       NULL']

# 3. Preprocessing

In [94]:
# df.hist(bins=30, figsize=(16,16))
# plt.show()

In [95]:
len(df)

79330

In [96]:
df.isna().sum().sum()

28

In [97]:
df = df.dropna()

In [98]:
df.isna().sum().sum()

0

In [100]:
len(df)

79302

In [33]:
# df.loc[df['Agent'] == '       NULL']

In [101]:
df.drop(columns=['ReservationStatusDate', 'Company'], inplace=True)

In [102]:
len(df)

79302

In [35]:
from sklearn.compose import ColumnTransformer

In [36]:
y = df[['IsCanceled']]
X = df.drop(columns=['IsCanceled','ArrivalDateYear'])

In [37]:
num_features = ['LeadTime',
            'StaysInWeekendNights',
            'StaysInWeekNights',
            'Adults',
            'Children',
            'Babies',
            'PreviousCancellations',
            'PreviousBookingsNotCanceled',
            'BookingChanges',
            'DaysInWaitingList',
            'ADR',
            'RequiredCarParkingSpaces',
            'TotalOfSpecialRequests']

df_num = X[num_features]

df_cat = X.drop(columns=num_features)

cat_features = list(df_cat.columns)

In [103]:
len(X)

79302

In [105]:
len(y)

79302

In [106]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import scipy.sparse

In [107]:
le = LabelEncoder()
X[cat_features] = X[cat_features].apply(
                            lambda col: le.fit_transform(col))

In [108]:
len(X)

79302

In [40]:
X[cat_features].head()

,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,ReservedRoomType,AssignedRoomType,DepositType,Agent,CustomerType,ReservationStatus
0,5,26,0,2,125,5,3,0,0,0,0,5,2,1
1,5,26,0,0,125,6,3,0,0,0,0,8,2,0
2,5,26,0,0,125,6,3,0,0,0,0,8,2,0
3,5,26,0,0,125,6,3,0,0,0,0,8,2,0
4,5,26,1,0,125,6,3,0,0,0,0,8,2,0


In [114]:
ohe = OneHotEncoder(categories='auto', drop='first')
X_ohe = ohe.fit_transform(X[cat_features])

In [115]:
feature_names = list(ohe.get_feature_names(cat_features))

In [116]:
X_ohe = pd.DataFrame.sparse.from_spmatrix(X_ohe)

In [44]:
X_ohe

,0,1,2,3,4,5,6,7,8,9,...,506,507,508,509,510,511,512,513,514,515
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79297,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
79298,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
79299,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
79300,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [118]:
X_ohe.columns = feature_names

In [122]:
X_ohe.isnull().sum().sum()

0

In [124]:
len(X)

79302

In [125]:
len(X_ohe)

79302

In [126]:
X_merged = pd.concat([X, X_ohe], axis=1)

In [127]:
len(X_merged)

79330

In [ ]:
# X_merged.reset_index(drop=True, inplace=True)

In [ ]:
# X.reset_index(drop=True, inplace=True)

In [ ]:
X_merged.head()

In [ ]:
X_null = X_merged[feature_names].isnull().any(axis=1)

In [ ]:
X_null.tail(30)

# 4. Modelling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import accuracy

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_merged, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=50)

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

In [ ]:
log_reg.score(X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree_clf = DecisionTreeClassifier(max_depth=1)
dtree_clf.fit(X_train, y_train)

In [ ]:
dtree_clf.score(X_train, y_train)